# Getting started

To start using Ibis, you need a Python environment with Ibis installed. Follow the <a href='../../backends/SQLite#install'>installation instructions for SQLite</a> to setup an environment.

Once you have your environment ready, to start using Ibis simply import the `ibis` module:

In [4]:
import ibis

To make it things easier in this tutorial, we will be using _Ibis interactive mode_. For production code, that will rarely be the case. More details on Ibis non-interactive (aka lazy) mode are covered in the third tutorial, _Expressions, lazy mode and logging queries_.

To set the interactive mode, use:

In [5]:
ibis.options.interactive = True

Next thing we need is to create a connection object. The connection defines where the data is stored and where the computations will be performed.

For a comparison to pandas, this is not the same as where the data is imported from (e.g. `pandas.read_sql`). pandas loads data into memory and performs the computations itself. Ibis won't load the data and perform any computation, but instead will leave the data in the backend defined in the connection, and will _ask_ the backend to perform the computations.

In this tutorial we will be using a SQLite connection for its simplicity (no installation is needed). But Ibis can work with many different backends, including big data systems, or GPU-accelerated analytical databases. As well as most common relational databases (PostgreSQL, MySQL,...).

Let's download the SQLite database from the `ibis-tutorial-data` GCS (Google Cloud Storage) bucket, then connect to it using `ibis`.

In [15]:
!curl -LsS -o $TEMPDIR/geography.db 'https://storage.googleapis.com/ibis-tutorial-data/geography.db'

In [16]:
import os
import tempfile

connection = ibis.sqlite.connect(
    os.path.join(tempfile.gettempdir(), 'geography.db')
)

Note that if you installed Ibis with `pip` instead of `conda`, you may need to install the SQLite backend separately with `pip install 'ibis-framework[sqlite]'`.

### Exploring the data

To list the tables in the `connection` object, we can use the `.list_tables()` method. If you are using Jupyter, you can see all the methods and attributes of the `connection` object by writing `connection.` and pressing the `<TAB>` key.

In [4]:
connection.list_tables()

['countries', 'gdp']

These two tables include data about countries, and about GDP by country and year.

The data from countries has been obtained from [GeoNames](https://www.geonames.org/countries/).
The GDP table will be used in the next tutorial, and the data has been obtained from the
[World Bank website](https://data.worldbank.org/indicator/NY.GDP.MKTP.CD).

Next, we want to access a specific table in the database. We can create a handler to the `countries` table with:

In [5]:
countries = connection.table('countries')

To list the columns of the `countries` table, we can use the `columns` attribute.

Again, Jupyter users can see all the methods and attributes of the `countries` object by typing `countries.` and pressing `<TAB>`.

In [6]:
countries.columns

['iso_alpha2',
 'iso_alpha3',
 'iso_numeric',
 'fips',
 'name',
 'capital',
 'area_km2',
 'population',
 'continent']

We can now access a sample of the data. Let's focus on the `name`, `continent` and `population` columns to start with. We can visualize the values of the columns with:

In [7]:
countries['name', 'continent', 'population']

                     name continent  population
0                 Andorra        EU       84000
1    United Arab Emirates        AS     4975593
2             Afghanistan        AS    29121286
3     Antigua and Barbuda        NA       86754
4                Anguilla        NA       13254
..                    ...       ...         ...
247                 Yemen        AS    23495361
248               Mayotte        AF      159042
249          South Africa        AF    49000000
250                Zambia        AF    13460305
251              Zimbabwe        AF    13061000

[252 rows x 3 columns]

The table is too big for all the results to be displayed, and we probably don't want to see all of them at once anyway. For this reason, just the beginning and the end of the results is displayed. Often, the number of rows will be so large that this operation could take a long time.

To check how many rows a table has, we can use the `.count()` method:

In [8]:
countries.count()

252

To fetch just a subset of the rows, we can use the `.limit(n)` method, where `n` is the number of samples we want. In this case we will fetch the first `3` countries from the table:

In [9]:
countries['name', 'continent', 'population'].limit(3)

                   name continent  population
0               Andorra        EU       84000
1  United Arab Emirates        AS     4975593
2           Afghanistan        AS    29121286

### Filters and order

Now that we've got an intuition of the data available in the table `countries`, we will extract some information from it by applying filters and sorting the data.

Let's focus on a single continent. We can see a list of unique continents in the table using the `.distinct()` method:

In [10]:
countries[['continent']].distinct()

0    EU
1    AS
2    NA
3    AF
4    AN
5    SA
6    OC
Name: continent, dtype: object

We will focus on Asia (`AS` in the table). We can identify which rows belong to Asian countries using the standard Python `==` operator:

In [11]:
countries['continent'] == 'AS'

0      False
1       True
2       True
3      False
4      False
       ...  
247     True
248    False
249    False
250    False
251    False
Name: tmp, Length: 252, dtype: bool

The result has a value `True` for rows where the condition is true, and the value `False` when it's not.

We can provide this expression to the method `.filter()`, and save the result in the variable `asian_countries` for future use.

In [12]:
asian_countries = countries['name', 'continent', 'population'].filter(
    countries['continent'] == 'AS'
)
asian_countries

                              name continent  population
0             United Arab Emirates        AS     4975593
1                      Afghanistan        AS    29121286
2                          Armenia        AS     2968000
3                       Azerbaijan        AS     8303512
4                       Bangladesh        AS   156118464
5                          Bahrain        AS      738004
6                           Brunei        AS      395027
7                           Bhutan        AS      699847
8          Cocos [Keeling] Islands        AS         628
9                            China        AS  1330044000
10                         Georgia        AS     4630000
11                       Hong Kong        AS     6898686
12                       Indonesia        AS   242968342
13                          Israel        AS     7353985
14                           India        AS  1173108018
15  British Indian Ocean Territory        AS        4000
16                            I

We can check how many countries exist in Asia (based on the information in the database) by using the `.count()` method we've already seen:

In [13]:
asian_countries.count()

51

Next, we want to find the most populated countries in Asia. To obtain them, we are going to sort the countries by the column `population`, and just fetch the first 10. To sort by a column in Ibis, we can use the `.order_by()` method:

In [14]:
asian_countries.order_by('population').limit(10)

                             name continent  population
0         Cocos [Keeling] Islands        AS         628
1  British Indian Ocean Territory        AS        4000
2                          Brunei        AS      395027
3                        Maldives        AS      395650
4                           Macao        AS      449198
5                          Bhutan        AS      699847
6                         Bahrain        AS      738004
7                           Qatar        AS      840926
8                          Kuwait        AS     2789132
9                            Oman        AS     2967717

This will return the least populated countries, since `.order_by` will by default order in ascending order (ascending order like in `1, 2, 3, 4`). This behavior is consistent with SQL `ORDER BY`.

To order in descending order we can use `ibis.desc()`:

In [15]:
asian_countries.order_by(ibis.desc('population')).limit(10)

          name continent  population
0        China        AS  1330044000
1        India        AS  1173108018
2    Indonesia        AS   242968342
3     Pakistan        AS   184404791
4   Bangladesh        AS   156118464
5        Japan        AS   127288000
6  Philippines        AS    99900177
7      Vietnam        AS    89571130
8       Turkey        AS    77804122
9         Iran        AS    76923300

This is the list of the 10 most populated countries based on the data from [GeoNames](https://www.geonames.org/).

To learn more about Ibis, continue to the next tutorial.